In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [31]:
def scrape_cards(max_page):
    cards = []
    for pg in range(max_page+1):
        url = f"https://scryfall.com/search?as=full&order=name&page={pg}&q=is%3Acommander&unique=cards"
        response = requests.get(url)

        if response.status_code == 200: # if we are successful
            print(f"Response sucessful. Scraping page {pg}")

            # scrape data - card image, name, mana value & images and whether it is double-sided.
            soup = BeautifulSoup(response.text, 'html.parser')

            # find card profiles, which contains all the card info
            card_profiles = soup.find_all('div', class_='card-profile')
            print("nr of card profiles", len(card_profiles))

            for card in card_profiles:
                # find image element
                image_element = card.find('img', class_='card')

                # find mana cost element
                mana_cost_element = card.find('span', class_='card-text-mana-cost')
                total_mana_cost = 0 # initialise mana cost variable

                # Count the number of non-generic mana
                for abbr in mana_cost_element.find_all('abbr', class_='card-symbol'):
                    mana_type = abbr['title']
                    if 'generic' in mana_type:
                        generic_value = int(abbr.get_text(strip=True, separator='').replace('{', '').replace('}', '')) # extract number
                        total_mana_cost += generic_value

                    else:
                        total_mana_cost += 1 # at the moment, we don't care what color it is, only the total cost

                # store relevant information
                card_data = {
                    'img_url' : image_element['src'],
                    'card_name' : image_element['title'],
                    'double-sided' : 'data-rotate' in image_element.attrs,
                    'cost' : total_mana_cost

                }

                # add to list of cards
                cards.append(card_data)

    return cards


In [32]:
cards = scrape_cards(1)

Response sucessful. Scraping page 0
nr of card profiles 20
Response sucessful. Scraping page 1
nr of card profiles 20


In [34]:
len(cards)
for card in cards:
    print(card)

{'img_url': 'https://cards.scryfall.io/large/front/e/b/eb363654-2004-4db8-bbd2-5b121da4f2a0.jpg?1681158256', 'card_name': 'A-Acererak the Archlich (Adventures in the Forgotten Realms #A-87)', 'double-sided': False, 'cost': 3}
{'img_url': 'https://cards.scryfall.io/large/front/b/4/b443504e-1b25-4565-bad7-2575826c7bb9.jpg?1682292303', 'card_name': 'A-Alrund, God of the Cosmos // A-Hakka, Whispering Raven (Kaldheim #A-40)', 'double-sided': True, 'cost': 5}
{'img_url': 'https://cards.scryfall.io/large/front/c/9/c9f1fc97-00c0-492b-a4a3-b179afc2f95d.jpg?1674058353', 'card_name': 'Abaddon the Despoiler (Warhammer 40,000 Commander #2)', 'double-sided': False, 'cost': 5}
{'img_url': 'https://cards.scryfall.io/large/front/3/9/396f9198-67b6-45d8-91b4-dc853bff9623.jpg?1674134856', 'card_name': "Abdel Adrian, Gorion's Ward (Commander Legends: Battle for Baldur's Gate #2)", 'double-sided': False, 'cost': 5}
{'img_url': 'https://cards.scryfall.io/large/front/4/b/4b68bc46-5591-44dd-becc-eca154066925.j

In [21]:
df = pd.DataFrame(cards)

In [22]:
display(df)

,img_url,card_name,double-sided,cost
0,https://cards.scryfall.io/large/front/e/b/eb36...,A-Acererak the Archlich (Adventures in the For...,False,0
1,https://cards.scryfall.io/large/front/b/4/b443...,"A-Alrund, God of the Cosmos // A-Hakka, Whispe...",True,0
2,https://cards.scryfall.io/large/front/c/9/c9f1...,"Abaddon the Despoiler (Warhammer 40,000 Comman...",False,0
3,https://cards.scryfall.io/large/front/3/9/396f...,"Abdel Adrian, Gorion's Ward (Commander Legends...",False,0
4,https://cards.scryfall.io/large/front/4/b/4b68...,Abomination of Llanowar (Kaldheim Commander #81),False,0
5,https://cards.scryfall.io/large/front/8/2/82db...,"Aboshan, Cephalid Emperor (Odyssey #58)",False,0
6,https://cards.scryfall.io/large/front/7/a/7ad7...,A-Bruenor Battlehammer (Adventures in the Forg...,False,0
7,https://cards.scryfall.io/large/front/4/0/40ee...,"Abuelo, Ancestral Echo (The Lost Caverns of Ix...",False,0
8,https://cards.scryfall.io/large/front/7/5/75f0...,"Ace, Fearless Rebel (Doctor Who #98)",False,0
9,https://cards.scryfall.io/large/front/d/d/dd52...,Acererak the Archlich (Adventures in the Forgo...,False,0
